Pydantic is one of the most important language in the data science field that is used for the data validation. Now what is the type valiadtion. Imagine you have this code
def insert_patients(name, age):
  print(name)
  print(age)

insert_patients("shaheer", 'twenty')

here the user gives the age in the string but it works while it shouldnt. So one soltion of this is that you use the type hinting by name: str.This will tell the user that the name should be in the string. But now the prpoblem is that the user is bull shit he still gives you the string. In this caser the type hinting will not gives you the error so what should we do now. Now the user will be moving to make the logic of the type checking by itself. It will work but will you make it for every work. Nah its not possible there will be alot of the boiler plate code. So this was the first problem.

Now the second main problem is some autistic person gives you  the negative age. Now there is no check so this will go into the data base. Now you have to type the logic for this thing too. Thats alot of the fuss so in this case you will use the pydantic the saviour of the boiler plate code.

Now the pydantic works in 3 parts
-> First you will make the model. In this model or a class you will define the fields that will be the part of your program. Then you have to make the define their types their variable constraints like if age is > 0. This model is also known as the schema.

-> In the second step you will make the object of the pydantic and send the your data to the object in form of the dictionary. then you will have the error if the data type dont match in the first place. Now the autistic can also your program.yipeeeeeeeeee!!!

-> In the third step you will send the checked data to the function you gonna use this in. Now your function dont have to take care for the type validation.